# 데이터 설명 : 은행의 전화 마케팅에 대해 고객의 반응 여부 (predict_proba)

In [99]:
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [2]:
import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

display(train.head())
display(test.head())
display(submission.head())

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown
2,52573,46,blue-collar,married,secondary,no,1728,yes,no,unknown,26,may,2,-1,0,unknown
3,50458,39,management,divorced,secondary,no,51,no,no,unknown,17,jun,2,-1,0,unknown
4,52272,31,services,single,tertiary,no,1626,no,no,unknown,31,jul,1,-1,0,unknown


,ID,predict
0,53608,0.0
1,51055,0.0
2,52573,0.0
3,50458,0.0
4,52272,0.0


# EDA

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12870 entries, 0 to 12869
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         12870 non-null  int64 
 1   age        12870 non-null  int64 
 2   job        12870 non-null  object
 3   marital    12870 non-null  object
 4   education  12870 non-null  object
 5   default    12870 non-null  object
 6   balance    12870 non-null  int64 
 7   housing    12870 non-null  object
 8   loan       12870 non-null  object
 9   contact    12870 non-null  object
 10  day        12870 non-null  int64 
 11  month      12870 non-null  object
 12  campaign   12870 non-null  int64 
 13  pdays      12870 non-null  int64 
 14  previous   12870 non-null  int64 
 15  poutcome   12870 non-null  object
 16  y          12870 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.7+ MB


In [ ]:
train.describe()

In [4]:
# 결측치 없음
print(train.isnull().sum())
print(test.isnull().sum())

ID           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64
ID           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
campaign     0
pdays        0
previous     0
poutcome     0
dtype: int64


In [9]:
# 카테고리 컬럼 nunique 동일
print(train.select_dtypes(include='object').nunique())
test.select_dtypes(include='object').nunique()

job          12
marital       3
education     4
default       2
housing       2
loan          2
contact       3
month        12
poutcome      4
y             2
dtype: int64


job          12
marital       3
education     4
default       2
housing       2
loan          2
contact       3
month        12
poutcome      4
dtype: int64

# 데이터 전처리

In [10]:
y = train.pop("y")
train.drop(columns='ID', inplace=True)
test.drop(columns='ID', inplace=True)

In [22]:
y.value_counts()

no     8903
yes    3967
Name: y, dtype: int64

In [23]:
le = LabelEncoder()
y = le.fit_transform(y)

In [14]:
# 숫자형 컬럼 중 스케일링 필요한 컬럼명만 선택
num_columns = ['balance', 'pdays']

In [12]:
scaler =  StandardScaler()

In [16]:
scaler.fit(train[num_columns])

StandardScaler()

In [17]:
train[num_columns] = scaler.transform(train[num_columns])
test[num_columns] = scaler.transform(test[num_columns])

In [19]:
# 카테고리형 컬럼 전처리
train = pd.get_dummies(train, drop_first=True)
test = pd.get_dummies(test, drop_first=True)

# 모델링

In [32]:
x_train, x_val, y_train, y_val = train_test_split(train, y, test_size=0.2, random_state=42, stratify=y)

In [95]:
rfc = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=20)

In [96]:
rfc.fit(x_train, y_train)

RandomForestClassifier(max_depth=20, random_state=42)

In [97]:
predict_rfc = rfc.predict(x_val)

In [98]:
auc_score_rfc = roc_auc_score(predict_rfc, y_val)
auc_score_rfc

0.7603866519130228

In [128]:
xgb = XGBClassifier(random_state=42, eval_metric='auc')

In [129]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [130]:
predict_xgb = xgb.predict(x_val)

In [131]:
auc_score_xgb = roc_auc_score(predict_xgb, y_val)
auc_score_xgb

0.7465334397702921

# 평가 및 제출

In [132]:
rfc_final = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=20)

In [133]:
rfc_final.fit(train, y)

RandomForestClassifier(max_depth=20, random_state=42)

In [134]:
predict_final = rfc_final.predict(test)

In [135]:
predict_final

array([1, 1, 0, ..., 0, 1, 0])

In [136]:
submission['predict'] = predict_final

In [137]:
submission['predict'].value_counts()

0    3437
1     854
Name: predict, dtype: int64

In [ ]:
#submission.to_csv('000000', index=False)